In [1]:
import pandas as pd
import numpy as np

# Use the correct relative paths from the root of your project folder
acs_path = '../data/ACS_Estimates_2010-2022.xlsx'
grade_enrollment_path = '../data/Bldg_Grade_Enroll_Cohort_Survival.xlsx'

# Load the data into Pandas DataFrames
acs_data = pd.read_excel(acs_path)
grade_enrollment_data = pd.read_excel(grade_enrollment_path)

# Display the first few rows of the DataFrames to verify
##grade_enrollment_data.head(1000)
acs_data.head(1000)

,Year,GEO,District,Total,In Households,Under 3,3_4,5,6-8,9-11,12-14,15-17,In Group Quarters,PK_Estimate,Elementary_Estimate,MS_Estimate,HS_Estimate
0,2010,9700000US2928860,"Springfield R-XII School District, Missouri",40689.0,40530.0,7332.0,5431.0,1783.0,7063.0,6157.0,7309.0,5455.0,159.0,5431,15003.0,7309.0,7255.0
1,2011,9700000US2928860,"Springfield R-XII School District, Missouri",38358.0,37960.0,7891.0,4152.0,3352.0,5434.0,6282.0,5810.0,5039.0,398.0,4152,15068.0,5810.0,6702.0
2,2012,9700000US2928860,"Springfield R-XII School District, Missouri",39045.0,38755.0,7424.0,5098.0,2634.0,6210.0,5648.0,6341.0,5400.0,290.0,5098,14492.0,6341.0,7182.0
3,2013,9700000US2928860,"Springfield R-XII School District, Missouri",41939.0,41644.0,7941.0,5600.0,2521.0,7014.0,6677.0,5118.0,6773.0,295.0,5600,16212.0,5118.0,9008.0
4,2014,9700000US2928860,"Springfield R-XII School District, Missouri",41738.0,41397.0,7568.0,4822.0,2637.0,6718.0,7365.0,6803.0,5484.0,341.0,4822,16720.0,6803.0,7294.0
5,2015,9700000US2928860,"Springfield R-XII School District, Missouri",40236.0,39960.0,7238.0,4956.0,1344.0,7144.0,6004.0,6645.0,6629.0,276.0,4956,14492.0,6645.0,8817.0
6,2016,9700000US2928860,"Springfield R-XII School District, Missouri",40188.0,39964.0,7081.0,4839.0,1821.0,7896.0,6138.0,6802.0,5387.0,224.0,4839,15855.0,6802.0,7165.0
7,2017,9700000US2928860,"Springfield R-XII School District, Missouri",39851.0,39602.0,6345.0,4376.0,3410.0,7906.0,5671.0,5985.0,5909.0,249.0,4376,16987.0,5985.0,7859.0
8,2018,9700000US2928860,"Springfield R-XII School District, Missouri",40333.0,40020.0,7607.0,4753.0,2438.0,5742.0,6562.0,6682.0,6236.0,313.0,4753,14742.0,6682.0,8294.0
9,2019,9700000US2928860,"Springfield R-XII School District, Missouri",41268.0,40712.0,5802.0,5214.0,1499.0,7103.0,6359.0,7700.0,7035.0,556.0,5214,14961.0,7700.0,9357.0


In [2]:
# Step 1: Calculate the sum of Elementary_Estimate, MS_Estimate, and HS_Estimate for each year in the ACS data
acs_data['RESIDENT_ESTIMATE'] = acs_data['Elementary_Estimate'] + acs_data['MS_Estimate'] + acs_data['HS_Estimate']

# Create a smaller DataFrame with just the 'Year' and 'Resident_Estimate' columns for merging
resident_estimate_by_year = acs_data[['Year', 'RESIDENT_ESTIMATE']].drop_duplicates()

# Manually set the 2020 Resident Estimate
resident_estimate_by_year.loc[resident_estimate_by_year['Year'] == 2020, 'RESIDENT_ESTIMATE'] = 37170

# Step 2: Merge this summed value into the grade_enrollment_data DataFrame
# Perform a left join on 'YEAR' from grade_enrollment_data to 'Year' in the resident_estimate_by_year DataFrame
grade_enrollment_data = pd.merge(grade_enrollment_data, resident_estimate_by_year, left_on='YEAR', right_on='Year', how='left')

# Drop the extra 'Year' column from the merge
grade_enrollment_data = grade_enrollment_data.drop(columns=['Year'])

# Step 3: Calculate the ENROLLMENT_RESIDENT_PROPORTION by dividing TOTAL_ENROLLMENT by RESIDENT_ESTIMATE
grade_enrollment_data['ENROLLMENT_RESIDENT_PROPORTION'] = np.where(grade_enrollment_data['RESIDENT_ESTIMATE'] != 0,
                                                         grade_enrollment_data['TOTAL_ENROLLMENT'] / grade_enrollment_data['RESIDENT_ESTIMATE'],
                                                         np.nan)

# Display the updated grade_enrollment_data DataFrame to verify the new column
grade_enrollment_data.head(1000)

,YEAR,COUNTY_DISTRICT_CODE,DISTRICT_NAME,SCHOOL_CODE,SCHOOL_NAME,GRADE,TOTAL_ENROLLMENT,COHORT_SURVIVAL_RATE,RESIDENT_ESTIMATE,ENROLLMENT_RESIDENT_PROPORTION
0,2006,39141,SPRINGFIELD R-XII,1050,CENTRAL HIGH,6,35,NaN,NaN,NaN
1,2006,39141,SPRINGFIELD R-XII,1050,CENTRAL HIGH,7,40,NaN,NaN,NaN
2,2006,39141,SPRINGFIELD R-XII,1050,CENTRAL HIGH,8,40,NaN,NaN,NaN
3,2006,39141,SPRINGFIELD R-XII,1050,CENTRAL HIGH,9,397,NaN,NaN,NaN
4,2006,39141,SPRINGFIELD R-XII,1050,CENTRAL HIGH,10,380,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
995,2011,39141,SPRINGFIELD R-XII,3080,PERSHING MIDDLE,6,239,1.039130,27580.0,0.008666
996,2011,39141,SPRINGFIELD R-XII,3080,PERSHING MIDDLE,7,229,0.982833,27580.0,0.008303
997,2011,39141,SPRINGFIELD R-XII,3080,PERSHING MIDDLE,8,236,1.123810,27580.0,0.008557
998,2011,39141,SPRINGFIELD R-XII,3080,PERSHING MIDDLE,0-8,704,1.046062,27580.0,0.025526


In [4]:
# Filter the dataset to include only rows where 'YEAR' is between 2010 and 2023
filtered_data = grade_enrollment_data[(grade_enrollment_data['YEAR'] >= 2010) & (grade_enrollment_data['YEAR'] <= 2023)]

# Display the first few rows of the filtered dataset to verify
print(filtered_data.head(1000))


      YEAR  COUNTY_DISTRICT_CODE      DISTRICT_NAME  SCHOOL_CODE  \
40    2010                 39141  SPRINGFIELD R-XII         1050   
41    2010                 39141  SPRINGFIELD R-XII         1050   
42    2010                 39141  SPRINGFIELD R-XII         1050   
43    2010                 39141  SPRINGFIELD R-XII         1050   
44    2010                 39141  SPRINGFIELD R-XII         1050   
...    ...                   ...                ...          ...   
1289  2015                 39141  SPRINGFIELD R-XII         3140   
1290  2016                 39141  SPRINGFIELD R-XII         3140   
1291  2016                 39141  SPRINGFIELD R-XII         3140   
1292  2016                 39141  SPRINGFIELD R-XII         3140   
1293  2016                 39141  SPRINGFIELD R-XII         3140   

       SCHOOL_NAME GRADE  TOTAL_ENROLLMENT  COHORT_SURVIVAL_RATE  \
40    CENTRAL HIGH     6                41              1.000000   
41    CENTRAL HIGH     7                43     

In [5]:
# Specify the path where you want to save the new Excel file
output_path = '../data/enrollment_training_data.xlsx'

# Save the DataFrame as an Excel file at the specified path
filtered_data.to_excel(output_path, index=False)

# Print a confirmation message
print(f"Dataset saved as '{output_path}'.")

Dataset saved as '../data/enrollment_training_data.xlsx'.


In [6]:
# Load the dataset from the saved Excel file
file_path = '../data/enrollment_training_data.xlsx'
data = pd.read_excel(file_path)

# Replace blank values with NaN
data.replace('', np.nan, inplace=True)

# Save the DataFrame back to the same Excel file
data.to_excel(file_path, index=False)

print(f"Blank values in '{file_path}' have been replaced with NaN.")

Blank values in '../data/enrollment_training_data.xlsx' have been replaced with NaN.
